In [1]:
import os
os.getcwd()
os.chdir('/work')
print(os.getcwd())
current_folder = globals()['_dh'][0]
import pathlib
import sys
del sys.argv[1]
print(sys.path)


from experiments.transformations.analysis.activation_distance.activation_distance_utils import *


/work
['/work/code/experiments/transformations/analysis/activation_distance_for_accuracy', '/work/scripts', '/work/code', '/work/code/experiments/transformations/analysis/activation_distance_for_accuracy', '/src', '/src/models', '/work/code/ML_framework', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/thedude/.ipython']


In [2]:
from sty import fg, bg, rs, ef
from experiments.transformations.analysis.activation_distance.activation_distance_utils import *
import torch
from ML_framework.distance_activation import DistanceActivation
from torch.utils.data import DataLoader
import os
from experiments.transformations.utils.datasets import  SameDifferentSampler
from ML_framework.datasets import add_compute_stats
import pickle
from torch.utils.data.sampler import WeightedRandomSampler
from experiments.transformations.utils.misc import classes_sets, exp_categorization_task, project_path, get_fulltrain_strings_shapenet
from experiments.transformations.utils.datasets import get_transforms, add_unpack_transf_info
import torchvision
##




class NwaySampler(SameDifferentSampler):
    def get_subclasses_weighted_sampler(self, dataset):
        weights_class = 1 / np.array([np.sum([True for k, v in dataset.subclasses_to_classes.items() if v == c]) for c in range(len(dataset.classes))])
        weights_dataset = [weights_class[dataset.subclasses_to_classes[i]] for i in dataset.subclasses_names]
        return WeightedRandomSampler(weights=weights_dataset, num_samples=len(dataset.subclasses_names), replacement=True)

    def __init__(self, dataset, N = 5):
        self.dataset = dataset
        self.N = N
        self.batch_size = 1
        self.subclasses = dataset.subclasses
        self.subclasses_names = dataset.subclasses_names
        self.sampler_training = self.get_subclasses_weighted_sampler(dataset)
        self.sampler_candidate = self.get_subclasses_weighted_sampler(dataset)

    def __iter__(self):
        training_idx = []
        candidate_idx = []
        for idx in self.sampler_training:
            sel_obj = self.subclasses_names[idx]
            training_idx.append(np.random.choice(self.subclasses[sel_obj], 1)[0])
            for c in range(self.N):
                if c == 0:  # the first one is the "same"
                    if 'v' in dataset.transform_str:
                        candidate_idx.append(np.random.choice(self.subclasses[sel_obj], 1)[0])
                    else:
                        candidate_idx.append(training_idx[-1])
                else:
                    random_candidate_idx = np.random.choice(len(self.subclasses_names))
                    candidate_idx.append(np.random.choice(self.subclasses[self.subclasses_names[random_candidate_idx]], 1)[0])

            yield np.hstack((training_idx, candidate_idx ))
            candidate_idx = []
            training_idx = []
#         yield np.hstack((training_idx, candidate_idx))


def change_transformation_dataset(dataset, transformations):
    # num_v = 50 if 'v' in transformations else 1
    add_PIL_transforms, add_T_transforms = get_transforms(transformations, info=False)
    dataset.transform_str = transformations

    dataset.transform = torchvision.transforms.Compose([*add_PIL_transforms, torchvision.transforms.ToTensor(), *add_T_transforms])
    normalize = torchvision.transforms.Normalize(mean=stats['mean'],
                                                 std=stats['std'])
    dataset.transform.transforms += [normalize]


def get_exp(max_objs):
    shared_args = lambda: dict(seed=1, project_name='Transformations', use_weblog=0, batch_size=64, max_epochs=-1 if torch.cuda.is_available() else 2, num_viewpoints_train=50, use_mat=0, max_objs_per_class_train=max_objs)
    exp = exp_categorization_task(**shared_args(),
                                  classes_set='set2',
                                  network_name='vgg11bn',
                                  use_device_num=2,
                                  verbose=False)
    print(dataset_name)

    if dataset_name == 'ShapeNet':
        name_folder = f'ShapeNet2D/ShapeNet2DFullNomat/train'
        classes_set = classes_sets[exp.classes_set]
        save_load = f'./tmp_{dataset_name}.sample'
    if dataset_name == 'ETHdark':
        name_folder = 'ETH-80-master/transformed_darkened'
        classes_set = None
        save_load = None
    if dataset_name == 'ETH':
        name_folder = 'ETH-80-master/transformed'
        classes_set = None
        save_load = None
    if dataset_name == 'ETHoriginal':
        name_folder = 'ETH-80-master/only_resized'
        classes_set = None
        save_load = None
        
    add_PIL_transforms, add_T_transforms = get_transforms('', info=False)
    args = dict(name_generator=f"train {exp.classes_set}",
                add_PIL_transforms=add_PIL_transforms,
                add_tensor_transforms=add_T_transforms,
                save_stats_file=None,
                save_load_samples_filename=save_load,
                stats=stats,
                num_objects_per_class=exp.max_objs_per_class_train,
                name_classes=classes_set,
                num_viewpoints_per_object=exp.num_viewpoints_train,
                root=f"./data/{name_folder}")
                
    dataset = add_compute_stats(SubclassImageFolder)(**args)
    loader = DataLoader(dataset, batch_sampler=NwaySampler(dataset=dataset, N=5), num_workers=0, timeout=0, pin_memory=True)

    return exp, dataset, loader

stats = {'mean': [0.06229676, 0.0607271, 0.05646703], 'std': [0.14454809, 0.14061172, 0.12994126]}

class SanityCheck(DistanceActivation):
    def get_base_and_other_canvasses(self, class_num, name_class):
        pass


In [7]:
def get_net(pt_transform, pt_objs=None, seed=0):
    pt_num_v = 50 if 'v' in pt_transform else 1

    if pt_transform == 'vanilla':
        exp.pretraining = 'vanilla'
        exp.network_name = network_name
        net = exp.get_net(10)
        net_name = 'vanilla'
        folder_output = 'others'

    elif pt_transform == 'ImageNet':
        exp.network_name = 'vgg11bn'
        exp.pretraining = './models/ptImageNet/vgg11bn.pt'
        net = exp.get_net(1000)
        net_name = 'ImageNet'
        folder_output = 'others'
    else:
        _, output_model, _, net_name = get_fulltrain_strings_shapenet(pt_transform, pt_objs, pt_num_v, class_set='set1', mat=0, network_name=network_name, seed=seed)
        print(net_name)
        exp.pretraining = output_model 
        exp.network_name = network_name
        folder_output = 'fulltrain'
        net = exp.get_net(new_num_classes=10)

        if network_name == 'samediffnet':
            net = net.backbone
    #     print(net)

    net.eval()
    net.cuda()
    return net, net_name

In [8]:
def do_it(transform, pt_transform, pt_obj=None, seed=0):
    try:
        net, net_name = get_net(pt_transform, pt_obj, seed)
    except AssertionError:
        print(f"FILE NOT FOUND!!! {pt_transform}, {pt_obj}, {network_name}")
        return None, None
    net.cuda()
#     print(net)
    change_transformation_dataset(dataset, transform)
    sc = SanityCheck(net, distance='cossim')
    corr = 0
    corr_images = 0
    tot = 0
    num_trials = 100

    
    for idx, data in enumerate(loader):
        img_set1, lb, lo = data
#         framework_utils.imshow_batch(img_set1, stats=stats)
        cossim_net, cossim_images, p = sc.get_cosine_similarity_from_images(img_set1[0], img_set1[1:])

        corr += np.argmax(cossim_net[sc.all_layers_name[-1]]) == 0
        corr_images += np.argmax(cossim_images) == 0
        tot += 1
        if tot == num_trials:
            break
    
    acc_net = corr/tot
    acc_img = corr_images/tot
    print(f"accuracy network: {acc_net}")
    print(f"accuracy image: {acc_img}")
    
    output_path = f'./results/transformations/activation_distance/{dataset_name}/{network_name}/{folder_output}/cossim_{transform}_{net_name}.pickle'
    print(fg.red + f"Saving result in {output_path}" + rs.fg)
    pathlib.Path(os.path.dirname(output_path)).mkdir(parents=True, exist_ok=True)

    pickle.dump({'acc_net': acc_net,
                 'acc_img': acc_img}, open(output_path,'wb'))


    return acc_net, acc_img

# Load Dataset ShapeNet

In [ ]:
dataset_name = 'ShapeNet'
exp, dataset, loader = get_exp(500)

# Or ETH

In [5]:
dataset_name = 'ETHoriginal'
exp, dataset, loader = get_exp(500)

Using cuda - you are probably on the server
Running experiment in SCRIPT mode. Command line arguments will have the precedence over keywords arguments.
Device Name: [GeForce RTX 3090] - Selected device num: 2

**LIST_TAGS**:
['ptvanilla', 'bkbnvanilla']
***PARAMS***
	add_cat_module : False
	additional_tags : None
	backbone_name : None
	batch_size : 64
	classes_set : set2
	device_name : GeForce RTX 3090
	diff_classes_set : None
	do_train : True
	force_cuda : None
	freeze_all_but_last : False
	freeze_backbone : False
	learning_rate : None
	max_epochs : -1
	max_iterations : -1
	max_objs_per_class_test : None
	max_objs_per_class_train : 500
	model_output_filename : None
	network_name : vgg11bn
	num_iterations_testing : -1
	num_runs : 1
	num_viewpoints_test : None
	num_viewpoints_train : 50
	output_filename : None
	patience_stagnation : -1
	pretraining : vanilla
	pretraining_backbone : vanilla
	project_name : Transformations
	scramble_fc : 0
	seed : 1
	stop_when_train_acc_is : -1
	transform

# Multi Nets + Multi Objects

In [9]:
folder_output = 'nway_test'

network_names = ["samediffnet", "resnet18", "resnet50", "vgg11bn", "vgg19bn", "alexnet", "googlenet", "densenet201"]

obj_list = [5, 50, 100, 250, 500]
seed = [1, 2, 3]
for network_name in network_names:
    for obj in obj_list:
        [do_it('v', 'v', obj, seed=s) for s in seed]
        [do_it('t', 't', obj, seed=s) for s in seed]
        [do_it('r', 'r', obj, seed=s) for s in seed]
        [do_it('s', 's', obj, seed=s) for s in seed]
        [do_it('b', 'b', obj, seed=s) for s in seed]
        [do_it('c', 'c', obj, seed=s) for s in seed]

        [do_it('v', '', obj, seed=s) for s in seed]
        [do_it('t', '', obj, seed=s) for s in seed]
        [do_it('r', '', obj, seed=s) for s in seed]
        [do_it('s', '', obj, seed=s) for s in seed]
        [do_it('b', '', obj, seed=s) for s in seed]
        [do_it('c', '', obj, seed=s) for s in seed]



FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1
Loading.. full model from ./models/transformations//ShapeNet/samediffnet/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1.pt... Done.
accuracy network: 0.8
accuracy image: 0.7125
Saving result in ./results/transformations/activation_distance/ETHoriginal/samediffnet/nway_test/cossim_v_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1.pickle
FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2
Loading.. full model from ./models/transformations//ShapeNet/samediffnet/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2.pt... Done.
accuracy network: 0.725
accuracy image: 0.6875
Saving result in ./results/transformations/activation_distance/ETHoriginal/samediffnet/nway_test/cossim_v_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2.pickle
FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S3
Loading.. full model from ./models/transformations//ShapeNet/samediffnet/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S3.pt... Done.
accuracy network: 0.7625
accuracy image: 0.675
Saving result in ./res

/opt/conda/lib/python3.8/site-packages/torchvision/models/googlenet.py:77: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


Loading.. full model from ./models/transformations//ShapeNet/googlenet/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1.pt... Done.
accuracy network: 0.6375
accuracy image: 0.6
Saving result in ./results/transformations/activation_distance/ETHoriginal/googlenet/nway_test/cossim_v_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1.pickle
FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2
Loading.. full model from ./models/transformations//ShapeNet/googlenet/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2.pt... Done.
accuracy network: 0.625
accuracy image: 0.55
Saving result in ./results/transformations/activation_distance/ETHoriginal/googlenet/nway_test/cossim_v_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2.pickle
FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S3
Loading.. full model from ./models/transformations//ShapeNet/googlenet/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S3.pt... Done.
accuracy network: 0.6875
accuracy image: 0.5625
Saving result in ./results/transformations/activation_distance/ETHoriginal

# Cross transformations

In [ ]:
dataset_name = 'ShapeNet'

folder_output = 'nway_test_cross'
seed = [1, 2, 3]

network_names = ['resnet18', 'vgg19bn',  'googlenet', 'densenet201', 'relationNet']
obj_list = [250]
for network_name in network_names:
    for obj in obj_list:
        [do_it('v', 't', obj, seed=s) for s in seed]
        [do_it('t', 'v', obj, seed=s) for s in seed]
        [do_it('t', 'r', obj, seed=s) for s in seed]
        [do_it('r', 's', obj, seed=s) for s in seed]
        [do_it('s', 'b', obj, seed=s) for s in seed]
        [do_it('b', 'c', obj, seed=s) for s in seed]
        [do_it('c', 'b', obj, seed=s) for s in seed]
        [do_it('t', 'b', obj, seed=s) for s in seed]
        [do_it('r', 'b', obj, seed=s) for s in seed]
        [do_it('v', 'b', obj, seed=s) for s in seed]
        [do_it('v', 'c', obj, seed=s) for s in seed]
        [do_it('t', 'c', obj, seed=s) for s in seed]
        [do_it('b', 't', obj, seed=s) for s in seed]
        [do_it('b', 'r', obj, seed=s) for s in seed]
        [do_it('c', 'v', obj, seed=s) for s in seed]
        [do_it('c', 's', obj, seed=s) for s in seed]
        [do_it('c', 't', obj, seed=s) for s in seed]
